In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
from random import randint
import pickle

In [7]:
movie_df = pd.read_pickle('top_movies_df.pickle')

In [8]:
links = movie_df['Link'].tolist()

In [9]:
links

['/title/tt4154796/?ref_=bo_cso_table_1',
 '/title/tt0499549/?ref_=bo_cso_table_2',
 '/title/tt0120338/?ref_=bo_cso_table_3',
 '/title/tt2488496/?ref_=bo_cso_table_4',
 '/title/tt4154756/?ref_=bo_cso_table_5',
 '/title/tt0369610/?ref_=bo_cso_table_6',
 '/title/tt0110357/?ref_=bo_cso_table_51',
 '/title/tt0848228/?ref_=bo_cso_table_8',
 '/title/tt2820852/?ref_=bo_cso_table_9',
 '/title/tt4520988/?ref_=bo_cso_table_10',
 '/title/tt2395427/?ref_=bo_cso_table_11',
 '/title/tt1825683/?ref_=bo_cso_table_12',
 '/title/tt1201607/?ref_=bo_cso_table_13',
 '/title/tt2527336/?ref_=bo_cso_table_14',
 '/title/tt4881806/?ref_=bo_cso_table_15',
 '/title/tt2294629/?ref_=bo_cso_table_16',
 '/title/tt2771200/?ref_=bo_cso_table_17',
 '/title/tt3606756/?ref_=bo_cso_table_18',
 '/title/tt4630562/?ref_=bo_cso_table_19',
 '/title/tt1300854/?ref_=bo_cso_table_20',
 '/title/tt2293640/?ref_=bo_cso_table_21',
 '/title/tt3498820/?ref_=bo_cso_table_22',
 '/title/tt1477834/?ref_=bo_cso_table_23',
 '/title/tt0167260/

In [12]:
#clean links by stripping away irrelevant variables
clean_links = []
for link in links:
    link= link.split('?', 1)[0].split('t', 4)[-1].split('/', 1)[0]
    clean_links.append(link)

In [26]:
#need to pass it a list of urls

def mojo_movie_dict(url_list):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - distributor
        - domestic opening $ 
        - MPAA rating
        - number of foreign markets released
        - IMDB link
    Return information as a dictionary.
    '''
    compiled_mojo = []
    
    base_url = 'https://www.boxofficemojo.com/title/tt{}'
    
    for url in url_list:
        #Create full url to scrape main page
        url = base_url.format(url)
        
        #Request HTML and parse
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,"lxml")

    
        headers = ['mojo_title', 'domestic_opening_gross',
                   'rating', 'foreign_market_count', 'genres', 'distributor',
                   'director', 'actor_1', 'actor_2', 'IMDB_link']
    
        #Get title
        title_string = soup.find(class_= 'a-size-extra-large').text
        mojo_title = title_string.split('(')[0].strip()
    
  
        #Get distribution company
        distributor= soup.find(class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile').find_all('span')[1].contents[0] 
    
        #Get domestic opening
        try:
            domestic_opening = soup.find(text='Domestic Opening').findNext().find(class_= 'money').text
        except:
            pass
    
        #Get rating
        try:
            rating = soup.find(text='MPAA').findNext().text
        
        except:
            pass 
    
        #find foreign markets, depending on format
        try: 
            if soup.find(text = 'Markets'):
                foreign_markets = soup.find('table', class_= 'a-bordered a-horizontal-stripes a-size-base-plus').find_all(class_='a-link-normal')[1].text[0:2]
    
            else:
                market_links = soup.find(class_='a-section mojo-h-scroll').find_all(class_= 'a-link-normal')
                foreign_markets = len(market_links)-1
        except:
            pass
        
        #find genres
        try:
            genres = soup.find(text='Genres').findNext().text
        except:
            pass
        
        #find IMDB movie page link and strip off reference parts of URL (everything after ?)
        try:
            link_string = soup.find(class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile').find_all('a', class_='a-link-normal')[-1]['href']
            link_string = link_string.split('?')[0].strip()
        except:
            pass
        
        #create url to scrape credits page
        credits_url = url+'/credits'
            
        #Request HTML and parse
        response = requests.get(credits_url)
        page = response.text
        credits_soup = BeautifulSoup(page,"lxml")
        
        #find director(first)
        
        director = credits_soup.find(id = 'principalCrew').find_all('tr')[1].text.split("\n",1)[0]
        
        #find principal actor
        actor_1 = credits_soup.find(id = 'principalCast').find_all('tr')[1].text.split("\n",1)[0]
        
        #find second actor
        actor_2 = actor_2 = credits_soup.find(id = 'principalCast').find_all('tr')[2].text.split("\n",1)[0]
    
        #Create movie dictionary and return
        mojo_dict = dict(zip(headers, [mojo_title,
                                domestic_opening,
                                rating, 
                                foreign_markets,
                                genres,
                                distributor,
                                director,
                                actor_1,
                                actor_2,
                                link_string]))
        
        compiled_mojo.append(mojo_dict)
        
        #random sleep counter 1-4 seconds to avoid being blocked while scraping
        time.sleep(randint(1,4))
    return compiled_mojo


In [27]:
#test_link = ['4154796','0499549','0120338','2488496','4154756','0369610']
mojo_compiled_dict = mojo_movie_dict(clean_links)


ReadTimeout: HTTPSConnectionPool(host='www.boxofficemojo.com', port=443): Read timed out. (read timeout=None)

In [ ]:
#pickle_out = open('mojo_compiled_dict.pickle', 'wb')
#pickle.dump(mojo_compiled_dict, pickle_out)
#pickle_out.close()

In [112]:
#df = pd.DataFrame(potential_df)

In [23]:
#mojo_compiled_dict

In [104]:
#df

,mojo_title,domestic_opening_gross,rating,foreign_market_count,genres,distributor,director,actor_1,actor_2,IMDB_link
0,Avengers: Endgame,"$357,115,007",PG-13,56,Action\n \n Adventure\n \n ...,Walt Disney Studios Motion Pictures,Anthony Russo,Robert Downey Jr.,Chris Evans,https://pro.imdb.com/title/tt4154796
1,Avatar,"$77,025,481",PG-13,77,Action\n \n Adventure\n \n ...,Twentieth Century Fox,James Cameron,Sam Worthington,Zoe Saldana,https://pro.imdb.com/title/tt0499549
2,Titanic,"$28,638,131",PG-13,67,Drama\n \n Romance,Paramount Pictures,James Cameron,Leonardo DiCaprio,Kate Winslet,https://pro.imdb.com/title/tt0120338
3,Star Wars: Episode VII - The Force Awakens,"$247,966,675",PG-13,62,Action\n \n Adventure\n \n ...,Walt Disney Studios Motion Pictures,J.J. Abrams,Daisy Ridley,John Boyega,https://pro.imdb.com/title/tt2488496
4,Avengers: Infinity War,"$257,698,183",PG-13,54,Action\n \n Adventure\n \n ...,Walt Disney Studios Motion Pictures,Anthony Russo,Robert Downey Jr.,Chris Hemsworth,https://pro.imdb.com/title/tt4154756
5,Jurassic World,"$208,806,270",PG-13,66,Action\n \n Adventure\n \n ...,Universal Pictures,Colin Trevorrow,Chris Pratt,Bryce Dallas Howard,https://pro.imdb.com/title/tt0369610
